In [39]:
import os
import math
import random
import numpy as np
import pandas as pd

from haversine import haversine
from geopy.distance import geodesic

In [40]:
ruta_excel = os.path.join("..", "data", "data.xlsx")
df = pd.read_excel(ruta_excel)
df.head(10)


,Zona,Nombre,Latitud,Longitud
0,Zona Cafe,Av. Epigmenio Gonzalez 702,20.647555,-100.384022
1,Zona Cafe,Cremerías La Granja 1973,20.647427,-100.399009
2,Zona Cafe,Divina Costura,20.647092,-100.404399
3,Zona Cafe,Del Pueblo 966,20.659303,-100.401885
4,Zona Cafe,Portal de la Alegría 59,20.650737,-100.396840
5,Zona Cafe,C. Mecánicos 4,20.637657,-100.396859
6,Zona Cafe,Av. Pascual Alcocer Vega 308,20.658224,-100.397958
7,Zona Cafe,Frutería El Güero,20.630758,-100.395272
8,Zona Cafe,Calle Cocineras,20.638676,-100.398766
9,Zona Cafe,Quintana Roo 5,20.656635,-100.381148


In [41]:
# Distancia Matriz
origin = df.loc[df['Zona'] == "Zona Origen"].to_dict(orient='records')[0]

In [42]:
method = "haversine"
print(len(df))

distance_matrix = np.zeros((len(df), len(df)))
for i in range(len(df)):
    i_df = df.loc[i]
    i_coord = (float(i_df['Latitud']), float(i_df['Longitud']))
    for j in range(len(df)):
        j_df = df.loc[j]
        j_coord = (float(j_df['Latitud']), float(j_df['Longitud']))
        dist = haversine(i_coord, j_coord) if method == "haversine" else geodesic(i_coord, j_coord).km
        distance_matrix[i, j] = dist
distance_matrix

107


array([[0.        , 1.55955597, 2.12089074, ..., 7.38674993, 6.93663985,
        7.3387631 ],
       [1.55955597, 0.        , 0.56200893, ..., 8.4206998 , 7.97889423,
        8.31140853],
       [2.12089074, 0.56200893, 0.        , ..., 8.80916624, 8.37212981,
        8.68080256],
       ...,
       [7.38674993, 8.4206998 , 8.80916624, ..., 0.        , 0.45057761,
        0.41070145],
       [6.93663985, 7.97889423, 8.37212981, ..., 0.45057761, 0.        ,
        0.5783763 ],
       [7.3387631 , 8.31140853, 8.68080256, ..., 0.41070145, 0.5783763 ,
        0.        ]], shape=(107, 107))

In [43]:
n = len(df)
distance_matrix = np.random.randint(1, 20, (n, n)).astype(float)
np.fill_diagonal(distance_matrix, np.inf)
feromone_matrix = np.ones((n, n))

alpha = 1
beta = 2
rho = 0.5

In [44]:
def probabs(current, visited):
    probs = []
    for j in range(n):
        if j not in visited:
            tau = feromone_matrix[current, j] ** alpha
            eta = (1 / distance_matrix[current, j]) ** beta
            probs.append(tau * eta)
        else:
            probs.append(0)
    probs = np.array(probs)
    probs = probs / probs.sum()
    return probs


In [45]:
def select_next_city(current, visited):
    probs = probabs(current, visited)
    return np.random.choice(range(n), p = probs)

In [46]:
def build_route(start=0):
    route = [start]
    while len(route) < n:
        next_city = select_next_city(route[-1], route)
        route.append(next_city)
    return route

In [47]:
def route_length(route):
    length = 0
    for i in range(len(route) - 1):
        length += distance_matrix[route[i], route[i + 1]]
    return length

In [48]:
def update_pheromones(routes):
    global feromone_matrix
    # Evaporación
    feromone_matrix *= (1 - rho)
    for r in routes:
        L = route_length(r)
        for i in range(len(r) - 1):
            a, b = r[i], r[i + 1]
            feromone_matrix[a, b] += 1 / L

In [49]:
for ciclo in range(1, 4):
    print(f"\n===== CICLO {ciclo} =====")
    hormigas = [build_route() for _ in range(3)]
    for idx, h in enumerate(hormigas, 1):
        print(f"Hormiga {idx}: {h}, Longitud = {route_length(h):.2f}")
    update_pheromones(hormigas)

print("\nMatriz final de feromonas:\n", np.round(feromone_matrix, 3))


===== CICLO 1 =====
Hormiga 1: [0, np.int64(76), np.int64(77), np.int64(25), np.int64(10), np.int64(97), np.int64(81), np.int64(93), np.int64(99), np.int64(21), np.int64(54), np.int64(16), np.int64(95), np.int64(88), np.int64(31), np.int64(9), np.int64(98), np.int64(105), np.int64(29), np.int64(1), np.int64(12), np.int64(70), np.int64(3), np.int64(80), np.int64(33), np.int64(2), np.int64(19), np.int64(34), np.int64(13), np.int64(43), np.int64(22), np.int64(86), np.int64(37), np.int64(44), np.int64(28), np.int64(82), np.int64(48), np.int64(68), np.int64(101), np.int64(71), np.int64(47), np.int64(52), np.int64(92), np.int64(102), np.int64(96), np.int64(100), np.int64(15), np.int64(49), np.int64(87), np.int64(73), np.int64(23), np.int64(17), np.int64(41), np.int64(74), np.int64(8), np.int64(50), np.int64(58), np.int64(69), np.int64(91), np.int64(4), np.int64(11), np.int64(36), np.int64(5), np.int64(89), np.int64(106), np.int64(59), np.int64(90), np.int64(60), np.int64(57), np.int64(32), 